# DINO: Self Distillation with No Labels

This notebook is the implementation for a model that is called DINO, created by the Facebook AI Research Team. DINO is a very useful model that learns image representations using contrastive learning, unlike traditional ViT, DINO is trained in a self supervised manner like Masked Autoencoders and SimCLR. The notebook compasses multiple topics like Dino, Dinov2 as well as using registers in transformers to show how they can help in the model performance.

Reference Papers: -
* [Emerging Properties in Self-Supervised Vision Transformers](https://arxiv.org/pdf/2104.14294)
* [DINOv2: Learning Robust Visual Features without Supervision](https://arxiv.org/pdf/2304.07193)
* [VISION TRANSFORMERS NEED REGISTERS](https://arxiv.org/pdf/2309.16588)

# DINO v1

## 1. Introduction

Dino is a methodoly proposed by researchers at Facebook AI Research, the main problem with how ViT's are traditionally trained is that they are trained using supervised datasets, that is they learn representations for images by using the loss that is created by the final MLP head which is used to predict the class the image belongs to. Although this gives rich representations it severely limits the models capability of learning as it needs a very large amount of labelled data. Transformers that are trained on text, during the pretraining stage train using a self supervised methodology as they simply try to predict the next word based on the previous n number of words in the sequence, since this does not require seperate labels, the transformer can fit on a much larger dataset, as since data does not need to be labelled there is a much larger availability of such data. 

Dino methodology aims to adress the problem by giving a method that allows the ViT to train in a self supervised manner which is called contrastive learning. Since it no longer needs labelled data, the weakness of needed labelled data no longer exists and just images themselves are enough to be given as input to the model.

## 2. Architecture and Methodology